<div class="alert alert-success">
    
## Машинное обучение

## Практическое домашнее задание 3




# Для выполнения домашнего задания №3:

В этом задании вам следует обучить две модели: Логистическая регрессия, RandomForest. По всем обученным моделям должен быть проведен подбор параметров, и, при необходимости, применены инструменты для борьбы с переобучением.
 В качестве основной метрики, с помощью которой следует сравнивать производительность моделей, использовать площадь под ROC-кривой (AUC-ROC).
 Перед выполнением всех экспериментов и выбора лучшей модели, определите как вы будете оценивать качество: на тестовых данных, которые могут быть получены с помощью кросс-валидации или предварительно отобранной тестовой выборки из таблицы `features.csv`.
Не забываем, что в любом случае подбор гиперпараметров должен проводиться на кроссвалидации. После того, как найдете лучшие гиперпараметры с помощью GridSearch, инициализируйте модель с найденными лучшими гиперпараметрами. 

Для решения задания вам требуется выполнить следующие шаги:

1. Проведите EDA, не забудьте масштабировать данные, если это требуется. (1 балл)
2. Оцените качество моделей с помощью кросс-валидации, подберите при этом лучшие параметры. (2 балла, каждый по 0.5 с учётом ответов на вопросы)
  - Как долго проводилась кросс-валидация каждой выбранной вами модели? Инструкцию по измерению времени можно найти ниже по тексту.
  - Какое качество при этом получилось? В данном задании мы используем метрику качества AUC-ROC.
  - Как оно соотносится с качеством другой модели? Чем вы можете объяснить эту разницу?
  - Быстрее ли работает, Случайный лес по сравнению с Логистической регрессией? Время замерьте с помощью кода ниже.

3. Постройте предсказания вероятностей получения положительного класса для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. модель не получилась константной).  (1 балл)

4. В качестве метрики использовать площадь под ROC-кривой (AUC-ROC). Построить матрицу ошибок (Confusion Matrix). (1 балл)
5. Как правило исходно в задачах автоматически используется порог (threshold) равный 0.5, но мы можем его менять. (3 балла)

    - Как увеличение порога отразится на precision/recall?  
    На практике, перед нами всегда встает trade-off: уменьшить полноту взамен на большую точность или получить больше ложно положительных примеров взамен на больший recall.
    - Постройте на одном графике precision, recall и f1 score в зависимости от порога.
    - Найдите порог, который соответствует максимизации f1 score.

6.  Провести интерпретацию признаков с помощью `_features_importances/permutation_importance`, постройте bar plot. На основе полученных результатов описать выводы. (2 балла)  


**Обратите внимание**: высокое качество работы на кросс-валидации (близкое к 100\%) — это в первую очередь повод задуматься о том, правильно ли вы обучаете модель. Например, возможно, вы заглядываете в будущее.

*Немного о метрике качества:*

 `AUC-ROC` — это метрика качества для алгоритма, выдающего оценки принадлежности первому классу. Все три алгоритма — случайный лес, SVM и логистическая регрессия — умеют выдавать такие оценки. Для этого нужно получать предсказания с помощью функции predict\_proba. Эта функция возвращает два столбца: первый содержит оценки принадлежности нулевому классу, второй — первому классу. Вам нужны значения из второго столбца:

```
pred = model.predict_proba(X_test)[:, 1]
```

**Измерение времени работы кода**
```  
import time
import datetime

start_time = datetime.datetime.now()

time.sleep(3) # place the piece of code to be measured here

print 'Time elapsed:', datetime.datetime.now() - start_time
```

**Рекомендации и советы**

Если все работает очень медленно:

  - Используйте для обучения и кросс-валидации не всю выборку, а некоторое ее подмножество — например, половину объектов. Подмножество лучше всего брать случайным, а не формировать его из первых m объектов.
  - Попробуйте упростить модель — например, уменьшить глубину деревьев в (max\_depth).

# Описание задачи

## Предметная область: Игра Dota 2

[Dota 2](https://ru.wikipedia.org/wiki/Dota_2) — многопользовательская компьютерная игра жанра [MOBA](https://ru.wikipedia.org/wiki/MOBA). Игроки играют между собой матчи. В каждом матче участвует две команды, 5 человек в каждой. Одна команда играет за светлую сторону (The Radiant), другая — за тёмную (The Dire). Цель каждой команды — уничтожить главное здание базы противника (трон).

Существуют [разные режимы игры](http://dota2.gamepedia.com/Game_modes/ru), мы будем рассматривать режим [Captain's Mode](http://dota2.gamepedia.com/Game_modes/ru#Captain.27s_Mode), в формате которого происходит большая часть киберспортивных мероприятий по Dota 2.

### Как проходит матч

#### 1. Игроки выбирают героев

Всего в игре чуть более 100 различных героев (персонажей). В начале игры, команды в определенном порядке выбирают героев себе и запрещают выбирать определенных героев противнику (баны). Каждый игрок будет управлять одним героем, в рамках одного матча не может быть несколько одинаковых героев.  Герои различаются между собой своими характеристиками и способностями. От комбинации выбранных героев во многом зависит успех команды.

![](http://imgur.com/XFr4HYE.jpg)

#### 2. Основная часть

Игроки могут получать золото и опыт за убийство чужих героев или прочих юнитов. Накопленный опыт влияет на уровень героя, который в свою очередь позволяет улучшать способности. За накопленное золото игроки покупают предметы, которые улучшают характеристики героев или дают им новые способности.

После смерти герой отправляется в "таверну" и возрождается только по прошествии некоторого времени, таким образом команда на некоторое время теряет игрока, однако игрок может досрочно выкупить героя из таверны за определенную сумму золота.

В течение игры команды развивают своих героев, обороняют свою часть поля и нападают на вражескую.

![](http://imgur.com/5b0SlQb.jpg)

#### 3. Конец игры

Игра заканчивается, когда одна из команд разрушет определенное число "башен" противника и уничтожает трон.

![](http://imgur.com/Du79Kzf.jpg)

Признаки, представленные в таблице features.csv, по мнению экспертов в предметной области являются наиболее важными для решения задачи предсказания победы команды. Тем не менее, не обязательно использовать эти признаки в исходном виде для применения методов машинного обучения — вы можете сделать новые признаки из имеющихся.

*Материал частично базируется на задаче по соревнованию Kaggle Dota 2 Win Probability Prediction, проводимом в рамках проекта курса по Машинному Обучению на платформе Coursera.*

### Пример чтения файла с признаками

In [ ]:
import pandas 

features = pandas.read_csv('./features.csv', index_col='match_id')

features.head()

##### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier"
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier"
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`
      

# Набор данных

Набор данных с матчами записан в файле matches.jsonlines.bz2. В каталоге dictionaries приведены расшифровки идентификаторов, которые присутствуют в записях матчей.

Чтение информации о матчах
Информация о матчах записана в сжатом текстовом файле matches.jsonlines.bz2, каждая строчка которого содержит объект в формате JSON. Запись в формате JSON преобразуется в python-объект при помощи стандартного модуля json. Пример чтения матчей:

In [7]:
import json

f = open("dictionaries/heroes.json")
data = json.load(f)
f.close()

data['14']

{'id': 14,
 'name': 'npc_dota_hero_pudge',
 'localized_name': 'Pudge',
 'primary_attr': 'str',
 'attack_type': 'Melee',
 'roles': ['Disabler', 'Initiator', 'Durable', 'Nuker'],
 'img': '/apps/dota2/images/heroes/pudge_full.png?',
 'icon': '/apps/dota2/images/heroes/pudge_icon.png',
 'base_health': 200,
 'base_health_regen': 1.5,
 'base_mana': 75,
 'base_mana_regen': 0.9,
 'base_armor': -1,
 'base_mr': 25,
 'base_attack_min': 27,
 'base_attack_max': 33,
 'base_str': 25,
 'base_agi': 14,
 'base_int': 14,
 'str_gain': 3.5,
 'agi_gain': 1.5,
 'int_gain': 1.5,
 'attack_range': 150,
 'projectile_speed': 0,
 'attack_rate': 1.7,
 'move_speed': 280,
 'turn_rate': 0.7,
 'cm_enabled': True,
 'legs': 2}